In [1]:
from time import sleep
import pyautogui
from os import path
from random import uniform, seed
import math
import pprint
from copy import deepcopy

In [2]:
seed()

In [3]:
ANIM_SECONDS_MIN = 0.01
ANIM_SECONDS_MAX = 0.3
CUR_DIR = "C:\\Users\\Shevis\\src\\github.com\\shevisj\\splitbuilder"
BUILD_MODE = "move_speed.png"
OBJECTS_MENU = "objects_menu.png"
TOOLS_MENU = "tools_menu.png"
MOVEMENT_MENU = "movement_menu.png"
ROTATION_MENU = "rotation_menu.png"
BLOCKS_MENU = "blocks_menu.png"
RAMPS_MENU = "blocks_ramps_menu.png"
FLOORS_MENU = "blocks_floors_menu.png"
GLASS_RAMP = "glass_ramp.png"
GLASS_FLOOR = "glass_floor.png"
ENVIRONMENT_MENU = "environment_menu.png"

In [4]:
X_POS = (216, 935)
Y_POS = (216, 1107)
Z_POS = (216, 1280)
SURF_RAMP_ANGLE = -31
DOWN_STRAIGHT_START_ROT = (-135, 0, -45)

# Types

In [5]:
class Vector3:
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = round(float(x), 2)
        self.y = round(float(y), 2)
        self.z = round(float(z), 2)
    
    def __repr__(self):
        return f"(x: {self.x}, y: {self.y}, z: {self.z})"

    def angle_to(self, other: 'Vector3'):
        pass

class Transform:
    def __init__(self, position: Vector3 = Vector3(), rotation: Vector3 = Vector3()):
        self.position = position
        self.rotation = rotation
    
    def forward(self, distance: float):
        pos_x = self.position.x - round(math.sin(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        pos_z = self.position.z - round(math.cos(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        return self.copy(
            position=Vector3(pos_x, self.position.y, pos_z)
        )

    def backward(self, distance: float):
        pos_x = self.position.x + round(math.sin(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        pos_z = self.position.z + round(math.cos(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        return self.copy(
            position=Vector3(pos_x, self.position.y, pos_z)
        )

    def left(self, distance: float):
        pos_x = self.position.x - round(math.cos(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        pos_z = self.position.z + round(math.sin(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        return self.copy(
            position=Vector3(pos_x, self.position.y, pos_z)
        )

    def right(self, distance: float):
        pos_x = self.position.x + round(math.cos(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        pos_z = self.position.z - round(math.sin(-1.0 * math.radians(self.rotation.y)) * distance, 2)
        return self.copy(
            position=Vector3(pos_x, self.position.y, pos_z)
        )
    
    def copy(self, position: Vector3 = None, rotation: Vector3 = None):
        new = deepcopy(self)
        if position is not None:
            new.position = position
        if rotation is not None:
            new.rotation = rotation
        return new

    def __repr__(self):
        return f"Transform(\n\tposition: {self.position},\n\trotation: {self.rotation},\n)"


In [6]:
t0 = Transform(rotation=Vector3(y=-45))

In [7]:
t0.left(500).position

(x: -353.55, y: 0.0, z: 353.55)

# Start Game

In [8]:
def find_icon(icon_name, no_print=False):
    # Find the game icon
    image_path = path.join(CUR_DIR, 'images', icon_name)
    try:
        game_icon = pyautogui.locateOnScreen(image_path, confidence=0.8)
        if game_icon:
            if not no_print:
                print(f"{icon_name} found: {game_icon}")
            return pyautogui.center(game_icon)
        return None
    except pyautogui.ImageNotFoundException:
        if not no_print:
            print("Unable to locate image file")
        return None

def click_icon(icon_name, no_print=False):
    game_icon = find_icon(icon_name=icon_name, no_print=no_print)
    if game_icon:
        # Start the game
        y_delta = int(game_icon.y + uniform(-20, 20))
        x_delta = int(game_icon.x + uniform(-20, 20))
        pyautogui.moveTo(x_delta, y_delta, duration=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX))
        pyautogui.click(game_icon)
    elif not no_print:
        print("Unable to locate game icon", icon_name)

def click_spot(x, y):
    y_delta = int(y + uniform(-20, 20))
    x_delta = int(x + uniform(-40, 40))
    pyautogui.moveTo(x_delta, y_delta, duration=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX))
    pyautogui.click(x=x_delta, y=y_delta)

# Movement

In [9]:
def walk_wasd(dur=2):
    pyautogui.keyDown('w')
    sleep(dur)
    pyautogui.keyUp('w')

# Building

In [55]:
def is_active(img):
    icon_path = path.join(CUR_DIR, 'images', img)
    icon = pyautogui.locateOnScreen(icon_path, confidence=0.8)
    return icon is not None

def activate_build_mode():
    if not is_active(BUILD_MODE):
        pyautogui.press('`', interval=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX), _pause=True)

def open_build_menu():
    if not is_active(OBJECTS_MENU):
        pyautogui.press('m', interval=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX), _pause=True)

def reset_build_menu():
    open_build_menu()
    click_icon(OBJECTS_MENU)
    for _ in range(3):
        if not is_active(ENVIRONMENT_MENU):
            pyautogui.press('escape', interval=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX), _pause=True)
            continue
        break

def enter_value(spot, val):
    click_spot(spot[0], spot[1])
    val_chars = list(str(val))
    for ch in val_chars:
        pyautogui.press(ch, interval=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX), _pause=True)
    pyautogui.press('enter', interval=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX), _pause=True)

def place_object_base(transform: Transform):
    click_icon(TOOLS_MENU)
    click_icon(MOVEMENT_MENU)
    enter_value(X_POS, transform.position.x)
    enter_value(Y_POS, transform.position.y)
    enter_value(Z_POS, transform.position.z)
    click_icon(ROTATION_MENU)
    enter_value(X_POS, transform.rotation.x)
    enter_value(Y_POS, transform.rotation.y)
    enter_value(Z_POS, transform.rotation.z)
    click_spot(1132, 1132)

def place_object(nav, transform: Transform):
    click_icon(OBJECTS_MENU)
    for menu in nav:
        click_icon(menu, no_print=True)
    place_object_base(transform)

def place_previous_object(transform: Transform):
    sleep(uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX))
    pyautogui.press('z', interval=uniform(ANIM_SECONDS_MIN, ANIM_SECONDS_MAX), _pause=True)
    place_object_base(transform)

def straight_panal_extrapolator(transform: Transform, n_panals):
    pos_x = transform.position.x
    pos_y = transform.position.y
    pos_z = transform.position.z
    rot_x = transform.rotation.x
    rot_y = transform.rotation.y
    rot_z = transform.rotation.z
    panal_width = 400
    sign = math.copysign(1, n_panals)
    panal_mag = abs(n_panals)
    def extrapolate():
        nonlocal pos_x, pos_z
        for _ in range(panal_mag - 1):
            pos_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y)) * panal_width * sign, 2)
            pos_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y)) * panal_width * sign, 2)
            yield Transform(
                position=Vector3(pos_x, pos_y, pos_z),
                rotation=Vector3(rot_x, rot_y, rot_z)
            )
    return extrapolate

def place_surf_straight(transform: Transform, n_panals):
    reset_build_menu()
    place_object(
        [OBJECTS_MENU, BLOCKS_MENU, FLOORS_MENU, GLASS_FLOOR],
        transform
    )
    spe = straight_panal_extrapolator(transform, n_panals=n_panals)
    for new_transform in spe():
        place_previous_object(
            new_transform
        )

def calc_sep_deg(radius, angle=45):
    return math.degrees(math.atan(400.0 / (float(radius) + (800 * math.cos(math.radians(abs(angle)))))))

# Radii is a list of 2-tuples with radius, count pairs
def flex_curv_extrapolator(transform: Transform, radii, sign=1, h_deltas=None, extra_sep=0.0):
    pos_x = transform.position.x
    pos_y = transform.position.y
    pos_z = transform.position.z
    rot_x = transform.rotation.x
    rot_y = transform.rotation.y
    rot_z = transform.rotation.z
    n_panals = sum([r[1] for r in radii])
    if h_deltas is not None:
        assert (isinstance(h_deltas, list))
        n_deltas = sum([r[1] for r in h_deltas])
        assert n_deltas == n_panals
        result = []
        for delta in h_deltas:
            result.extend([delta[0]] * delta[1])
        h_deltas = result
    else:
        h_deltas = [0] * n_panals
    def extrapolate():
        cur_pos_x = pos_x
        cur_pos_z = pos_z
        cur_rot_y = rot_y
        cur_pos_y = pos_y
        i = -1
        for cur_radius in radii:
            abs_cur_rad = abs(cur_radius[0])
            cur_center_x = cur_pos_x + round(math.sin(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0] * -1.0, 2)
            cur_center_z = cur_pos_z + round(math.cos(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0] * -1.0, 2)
            deg_offset = (extra_sep + calc_sep_deg(abs_cur_rad, rot_z)) * sign
            for _ in range(cur_radius[1]):
                i += 1
                cur_pos_y = cur_pos_y + h_deltas[i]
                cur_rot_y = mod_rot(round(cur_rot_y + deg_offset, 2))
                cur_pos_x = cur_center_x + round(math.sin(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0], 2)
                cur_pos_z = cur_center_z + round(math.cos(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0], 2)
                yield Transform(
                    position=Vector3(cur_pos_x, cur_pos_y, cur_pos_z),
                    rotation=Vector3(rot_x, cur_rot_y, rot_z)
                )
    return extrapolate

def place_flex_curv(transform: Transform, radii, sign=1, h_deltas=None, ignore_first=False, extra_sep=0.0):
    reset_build_menu()
    if not ignore_first:
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, FLOORS_MENU, GLASS_FLOOR],
            transform
        )
    fce = flex_curv_extrapolator(transform, radii, sign, h_deltas, extra_sep)
    first = True
    for new_transform in fce():
        if ignore_first and first:
            place_object(
                [OBJECTS_MENU, BLOCKS_MENU, FLOORS_MENU, GLASS_FLOOR],
                new_transform
            )
        else:
            place_previous_object(
                new_transform
            )
        first = False

def mod_rot(rot):
    currot = rot
    while currot < -180.0:
        currot += 360.0
    return round(((rot + 180.0) % 360.0) - 180.0, 2)
        

def curv_extrapolator(transform: Transform, radius, n_panals=5, h_delta=0, r_delta=0, extra_sep=0.0):
    pos_x = transform.position.x
    pos_y = transform.position.y
    pos_z = transform.position.z
    rot_x = transform.rotation.x
    rot_y = transform.rotation.y
    rot_z = transform.rotation.z
    r_sign = math.copysign(1.0, radius)
    abs_radius = abs(radius)
    sign = math.copysign(1.0, n_panals)
    abs_n_panals = abs(n_panals)
    center_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0 * r_sign, 2)
    center_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0 * r_sign, 2)
    def extrapolate():
        deg_offset = 0
        cur_pos_y = pos_y
        cur_abs_radius = abs_radius
        for _ in range(abs_n_panals - 1):
            
            cur_pos_y += h_delta
            cur_abs_radius += r_delta

            deg_offset += (extra_sep + calc_sep_deg(cur_abs_radius, rot_z)) * sign
            new_x = center_x + round(math.sin(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius * r_sign, 2)
            new_z = center_z + round(math.cos(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius * r_sign, 2)
            yield Transform(
                position=Vector3(new_x, cur_pos_y, new_z),
                rotation=Vector3(rot_x, mod_rot(round(rot_y + deg_offset, 2)), rot_z)
            )
    return extrapolate

def place_curv(transform: Transform, radius: int, n_panals=5, h_delta=0, r_delta=0, ignore_first=False):
    reset_build_menu()
    if not ignore_first:
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, FLOORS_MENU, GLASS_FLOOR],
            transform
        )
    ce = curv_extrapolator(transform, radius, n_panals, h_delta, r_delta)
    first = True
    for new_transform in ce():
        if ignore_first and first:
            place_object(
                [OBJECTS_MENU, BLOCKS_MENU, FLOORS_MENU, GLASS_FLOOR],
                new_transform
            )
        else:
            place_previous_object(
                new_transform
            )
        first = False
    

# Arbitrary Circles

In [11]:
def arb_curv_extrapolator(transform: Transform, radius, n_panals=5, h_delta=0, r_delta=0, deg_sep=0.0):
    pos_x = transform.position.x
    pos_y = transform.position.y
    pos_z = transform.position.z
    rot_x = transform.rotation.x
    rot_y = transform.rotation.y
    rot_z = transform.rotation.z
    r_sign = math.copysign(1.0, radius)
    abs_radius = abs(radius)
    sign = math.copysign(1.0, n_panals)
    abs_n_panals = abs(n_panals)
    center_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0 * r_sign, 2)
    center_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0 * r_sign, 2)
    def extrapolate():
        deg_offset = 0
        cur_pos_y = pos_y
        cur_abs_radius = abs_radius
        for _ in range(abs_n_panals - 1):
            
            cur_pos_y += h_delta
            cur_abs_radius += r_delta

            deg_offset += deg_sep * sign
            new_x = center_x + round(math.sin(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius * r_sign, 2)
            new_z = center_z + round(math.cos(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius * r_sign, 2)
            yield Transform(
                position=Vector3(new_x, cur_pos_y, new_z),
                rotation=Vector3(rot_x, mod_rot(round(rot_y + deg_offset, 2)), rot_z)
            )
    return extrapolate

def place_arb_curv(transform: Transform, radius: int, n_panals=5, h_delta=0, r_delta=0, deg_sep=0.0):
    reset_build_menu()
    ce = arb_curv_extrapolator(transform, radius, n_panals, h_delta, r_delta, deg_sep=deg_sep)
    for new_transform in ce():
        place_previous_object(
            new_transform
        )

# Composite structures

In [12]:
roots = [
    Transform(
        position=Vector3(0, 19000, 19800),
        rotation=Vector3(0, 0, 0)
    ),
    Transform(
        position=Vector3(11638, 19000, 16019),
        rotation=Vector3(0, -36, 0)
    ),
    Transform(
        position=Vector3(18831, 19000, 6119),
        rotation=Vector3(0, -72, 0)
    ),
    Transform(
        position=Vector3(18831, 19000, -6119),
        rotation=Vector3(0, -108, 0)
    ),
    Transform(
        position=Vector3(11638, 19000, -16019),
        rotation=Vector3(0, -144, 0)
    ),
    Transform(
        position=Vector3(0, 19000, -19800),
        rotation=Vector3(0, 180, 0)
    ),
    Transform(
        position=Vector3(-11638, 19000, -16019),
        rotation=Vector3(0, 144, 0)
    ),
    Transform(
        position=Vector3(-18831, 19000, -6119),
        rotation=Vector3(0, 108, 0)
    ),
    Transform(
        position=Vector3(-188310, 19000, 6119),
        rotation=Vector3(0, 72, 0)
    ),
    Transform(
        position=Vector3(-11638, 19000, 16019),
        rotation=Vector3(0, 36, 0)
    ),
]

def build_whirlwind_arm(root: Transform):
    reset_build_menu()
    # Place root
    place_object(
        [BLOCKS_MENU, FLOORS_MENU, GLASS_FLOOR],
        root
    )

    # Section A
    section_a_transform = root.forward(2000)
    section_a_transform.position.y -= 1000
    section_a_transform.rotation = Vector3(root.rotation.x, root.rotation.y - 36, -45)
    
    place_curv(
        section_a_transform,
        10000,
        3,
        -200,
        -198
    )

    section_a_transform_upper = section_a_transform.left(565)
    section_a_transform_upper.position.y += 566
    place_curv(
        section_a_transform_upper,
        10566,
        3,
        -200,
        -198
    )

    #place_flex_curv(
    #    Transform(
    #        position=section_a_start_pos,
    #        rotation=Vector3(root.rotation.x, root.rotation.y, -31)
    #    ),
    #   [
    #        (5000, 1),
    #        (4000, 2),
    #        (3000, 3),
    #        (4000, 2),
    #        (5000, 1),
    #    ],
    #    1,
    #    [
    #        (-50, 9)
    #    ]
    #)

def relative_position_test(start: Transform):
    reset_build_menu()
    place_object(
        [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_FLOOR],
        start
    )
    for i in range(0, 360, 15):
        start.rotation.y = mod_rot(i)
        place_object(
            [GLASS_FLOOR],
            start.left(2000 + i)
        )


# Runner

In [62]:
try:
    click_icon('game_icon.png')
    activate_build_mode()
    # place_flex_curv(
    #     transform=Transform(
    #         position=Vector3(17800, 18400, 18300),
    #         rotation=Vector3(0, 135, -31)
    #     ),
    #     radii=[
    #        (-5000, 1),
    #        (-4000, 2),
    #        (-3000, 3),
    #        (-4000, 2),
    #        (-5000, 1),
    #     ],
    #     sign=-1,
    #     h_deltas=[(0, 9)],
    #     ignore_first=True
    #     # [
    #     #    (-10, 1),
    #     #    (-15, 2),
    #     #    (-20, 3),
    #     #    (-25, 2),
    #     #    (-30, 1),
    #     # ]
    # )
    # place_curv(
    #    transform=Transform(
    #         position=Vector3(-15200, 11080, 17600),
    #         rotation=Vector3(0, -135, -45)
    #     ),
    #    radius=3500,
    #    n_panals=11,
    #    h_delta=-50,
    #    r_delta=-50,
    #    ignore_first=False
    # )
    # place_curv(
    #     transform=Transform(
    #         position=Vector3(17600, 11080, -15200),
    #         rotation=Vector3(0, 45, 45)
    #     ),
    #    radius=-3500,
    #    n_panals=-11,
    #    h_delta=-50,
    #    r_delta=-50,
    #    ignore_first=True
    # )

    place_flex_curv(
        transform=Transform(
            position=Vector3(600, 14600, 3600),
            rotation=Vector3(0, -68.01, -45)
        ),
        radii=[
           (4000, 1),
           (3000, 2),
           (2000, 3),
           (3000, 2),
           (4000, 1),
        ],
        sign=1,
        h_deltas=[(0, 9)],
        ignore_first=False,
        #extra_sep=5.0
    )
    place_flex_curv(
        transform=Transform(
            position=Vector3(3600, 14600, 600),
            rotation=Vector3(0, -21.99, 45)
        ),
        radii=[
           (-4000, 1),
           (-3000, 2),
           (-2000, 3),
           (-3000, 2),
           (-4000, 1),
        ],
        sign=-1,
        h_deltas=[(0, 9)],
        ignore_first=True,
        #extra_sep=5.0
    )


    # place_arb_curv(
    #     transform=Transform(
    #         position=Vector3(-5560.73, 16250, 6228.47),
    #         rotation=Vector3(0, 63, 0)
    #     ),
    #    radius=-2950,
    #    n_panals=18,
    #    deg_sep=18
    # )
except KeyboardInterrupt as e:
    print("Exiting")

game_icon.png found: Box(left=1408, top=2115, width=30, height=20)
objects_menu.png found: Box(left=315, top=371, width=144, height=24)
objects_menu.png found: Box(left=315, top=371, width=144, height=24)
tools_menu.png found: Box(left=557, top=371, width=104, height=24)
movement_menu.png found: Box(left=401, top=572, width=142, height=20)
rotation_menu.png found: Box(left=628, top=572, width=125, height=20)
tools_menu.png found: Box(left=557, top=371, width=104, height=24)
movement_menu.png found: Box(left=401, top=572, width=142, height=20)
rotation_menu.png found: Box(left=628, top=572, width=125, height=20)
tools_menu.png found: Box(left=557, top=371, width=104, height=24)
movement_menu.png found: Box(left=401, top=572, width=142, height=20)
rotation_menu.png found: Box(left=628, top=572, width=125, height=20)
tools_menu.png found: Box(left=557, top=371, width=104, height=24)
movement_menu.png found: Box(left=401, top=572, width=142, height=20)
rotation_menu.png found: Box(left=628

In [25]:
image = pyautogui.screenshot()
image.save('testing.png')

In [ ]:
ep = straight_panal_extrapolator((0, 0, 0), (0, 0, 0), 4)

In [ ]:
for p, r in ep():
    print(p, r)

(400, 0, 0) (0, 0, 0)
(800, 0, 0) (0, 0, 0)
(1200, 0, 0) (0, 0, 0)


In [ ]:
curv_extrapolator((0, 5000, 0), (0, -90, -31), 1000, 5)

0 5000 -1000


In [ ]:
ce = curv_extrapolator((-1546, 12890, 1235), (0, 0, -31), 1000, 5)

In [ ]:
print((-1546, 12890, 1235), (0, 0, -31))
for loc, rot in ce():
    print(loc, rot)

(-1546, 12890, 1235) (0, 0, -31)
(-1542, 12890, 1148) (0, 5.0, -31)
(-1531, 12890, 1061) (0, 10.0, -31)
(-1512, 12890, 976) (0, 15.0, -31)
(-1486, 12890, 893) (0, 20.0, -31)


In [ ]:
math.degrees(math.atan(400.0 / float(2000)))

11.309932474020215

In [24]:
800 * math.cos(math.radians(45))

565.685424949238

In [ ]:
math.cos(math.radians(31))

0.8571673007021123

In [74]:
mod_rot(-66666)

-66

In [85]:
fce = flex_curv_extrapolator((-1546, 12890, 1235), (0, 0, -31), [(1000, 2), (500, 3), (1000, 2)])

In [87]:
print((-1546, 12890, 1235), (0, 0, -31))
for loc, rot in fce():
    print(loc, rot)

(-1546, 12890, 1235) (0, 0, -31)
(-1516, 12890, 993) (0, 14, -31)
(-1277, 12890, 553) (0, 43, -31)
(-1130, 12890, 445) (0, 64, -31)
(-782, 12890, 411) (0, 105, -31)
(-424, 12890, 782) (0, 167, -31)
(-398, 12890, 1024) (0, -179, -31)
(-532, 12890, 1507) (0, -150, -31)


In [21]:
for i in range(-180, 180, 36):
    print(i)

-180
-144
-108
-72
-36
0
36
72
108
144


In [39]:
r = roots[0]

In [40]:
r

Transform(
	position: (x: 0, y: 1900, z: 19800),
	rotation: (x: 0, y: 0, z: 0),
)

In [41]:
r.forward(1000)

(x: 0, y: 1900, z: 20800)

In [24]:
(800 * math.cos(math.radians(45)))

565.685424949238

In [61]:
root = roots[2]

In [73]:
section_a_transform = root.forward(2000)
section_a_transform.position.y -= 1000
section_a_transform.rotation = Vector3(root.rotation.x, root.rotation.y - 36, -31)

In [65]:
section_a_transform.position = Vector3(y=1000)

In [74]:
section_a_transform

Transform(
	position: (x: 16929, y: 18000, z: 5501),
	rotation: (x: 0, y: -108, z: -31),
)

In [75]:
section_a_transform.left(565)

Transform(
	position: (x: 16754, y: 18000, z: 6038),
	rotation: (x: 0, y: -108, z: -31),
)

In [6]:
import numpy as np
import math as m
  
def Rx(theta):
  return np.matrix([[ 1, 0           , 0           ],
                   [ 0, m.cos(theta),-m.sin(theta)],
                   [ 0, m.sin(theta), m.cos(theta)]])
  
def Ry(theta):
  return np.matrix([[ m.cos(theta), 0, m.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-m.sin(theta), 0, m.cos(theta)]])
  
def Rz(theta):
  return np.matrix([[ m.cos(theta), -m.sin(theta), 0 ],
                   [ m.sin(theta), m.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])

In [7]:
phi = m.pi/2
theta = m.pi/4
psi = m.pi/2
print("phi =", phi)
print("theta  =", theta)
print("psi =", psi)
  
  
R = Rz(psi) * Ry(theta) * Rx(phi)
print(np.round(R, decimals=2))

phi = 1.5707963267948966
theta  = 0.7853981633974483
psi = 1.5707963267948966
[[ 0.   -0.    1.  ]
 [ 0.71  0.71 -0.  ]
 [-0.71  0.71  0.  ]]


In [8]:
import sys
tol = sys.float_info.epsilon * 10
  
if abs(R.item(0,0))< tol and abs(R.item(1,0)) < tol:
   eul1 = 0
   eul2 = m.atan2(-R.item(2,0), R.item(0,0))
   eul3 = m.atan2(-R.item(1,2), R.item(1,1))
else:   
   eul1 = m.atan2(R.item(1,0),R.item(0,0))
   sp = m.sin(eul1)
   cp = m.cos(eul1)
   eul2 = m.atan2(-R.item(2,0),cp*R.item(0,0)+sp*R.item(1,0))
   eul3 = m.atan2(sp*R.item(0,2)-cp*R.item(1,2),cp*R.item(1,1)-sp*R.item(0,1))
  
print("phi =", eul1)
print("theta  =", eul2)
print("psi =", eul3)

phi = 1.5707963267948966
theta  = 0.7853981633974483
psi = 1.5707963267948966


In [36]:
math.sqrt((400**2)*2) + 2500

3065.685424949238

In [98]:
math.sqrt((8.4825**2)*2)

11.996066542829778

In [100]:
math.sqrt((8.48**2)/2)

5.996265504461923

In [33]:
80000/2

40000.0

In [25]:
math.sqrt(12.5)

3.5355339059327378

In [26]:
5.65*2

11.3

In [68]:
round(2.3345, 2)

2.33

In [73]:
-90 + 42.15

-47.85

In [15]:
math.degrees(math.tan(0.5))

31.300827005537716

In [19]:
25/2

12.5

In [27]:
(3.5355339059327378*2) + (8.48*2)

24.031067811865476

In [32]:
3.5355339059327378*2

7.0710678118654755

In [2]:
(6.37+11.31)-4.2

13.48

In [3]:
13.48/2

6.74

In [4]:
18.99289 / 2

9.496445

In [27]:
14.14-11.31

2.83

In [28]:
11.31 - 2.83

8.48

In [33]:
13.43 + 8.48

21.91

In [71]:
8.48528137423857 / 2

4.242640687119285

In [42]:
math.sqrt((400**2)/2)

282.842712474619

In [44]:
math.sqrt(282.84**2 + 282.84**2)

399.9961639816062

In [47]:
math.cos(math.radians(90)) * 400

2.4492935982947064e-14

In [72]:
3.54+3.54+11.31

18.39

In [73]:
8.54+4.2

12.739999999999998

In [74]:
18.39 - 12.739999999999998

5.650000000000002

In [94]:
7.77 + 11.31 + 4.242640687119285

23.322640687119282

In [93]:
11.31 + 4.242640687119285

15.552640687119286

In [83]:
5.656854249492381/2

2.8284271247461903

In [84]:
11.31/2

5.655

In [88]:
21.91+(11.31 - 2.8275)

30.3925

In [87]:
5.655/2

2.8275

In [90]:
33.22 - (11.31 - 2.8275)

24.737499999999997

In [92]:
8.4825 / 2

4.24125

In [95]:
8.4825 + 4.24125

12.723749999999999

In [99]:
4.242640687119285 / 2

2.1213203435596424

In [102]:
24.74/4

6.185

In [103]:
(11.31 - 2.8275)

8.4825

In [114]:
list(arb_curv_extrapolator(Transform(), 200, 5, 0, 0, deg_sep=10.0)())

[Transform(
 	position: (x: 3.04, y: 0.0, z: -34.73),
 	rotation: (x: 0.0, y: 10.0, z: 0.0),
 ),
 Transform(
 	position: (x: 12.06, y: 0.0, z: -68.4),
 	rotation: (x: 0.0, y: 20.0, z: 0.0),
 ),
 Transform(
 	position: (x: 26.79, y: 0.0, z: -100.0),
 	rotation: (x: 0.0, y: 30.0, z: 0.0),
 ),
 Transform(
 	position: (x: 46.79, y: 0.0, z: -128.56),
 	rotation: (x: 0.0, y: 40.0, z: 0.0),
 )]

In [ ]:
t = Transform(
    position=Vector3(
        x=14900,
        y=12800,
        z=-12700,
    ),
    rotation=Vector3(
        x=-45,
        y=40,
        z=-25,
    ),
)
t2 = Transform(
    position=Vector3(
        x=15500,
        y=12800,
        z=-12000,
    ),
    rotation=Vector3(
        x=45,
        y=40,
        z=-25,
    ),
)